In [16]:
#import ppa
import ppa
import numpy as np
import matplotlib.pyplot as plt
from PTMCMCSampler.PTMCMCSampler import PTSampler
from chainconsumer import ChainConsumer
from num2tex import num2tex
import json
import scipy.stats as ss
import corner
import os
    
PSR_DICT_LIST = ppa.Load_All_Pulsar_Info()
PSR_NAME_LIST = list(PSR_DICT_LIST.keys())
PSR_DICT_LIST

{'J0437-4715': {'PSR': 'J0437-4715',
  'RAJ': '04:37:15.8961737',
  'DECJ': '-47:15:09.110714',
  'DTE_DM': 0.156,
  'PX': 6.37,
  'PX_ERR': 0.09000000000000002,
  'DATA': {'10cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/Data/J0437-4715_10cm.txt',
   '20cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/Data/J0437-4715_20cm.txt'},
  'RM': {'10cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/ionFR_correction/J0437-4715_ionFR_10cm.txt',
   '20cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/ionFR_correction/J0437-4715_ionFR_20cm.txt'}},
 'J0613-0200': {'PSR': 'J0613-0200',
  'RAJ': '06:13:43.975901',
  'DECJ': '-02:00:47.23742',
  'DTE_DM': 1.024,
  'PX': 1.01,
  'PX_ERR': 0.09000000000000002,
  'DATA': {'10cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/Data/J0613-0200_10cm.txt',
   '20cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/Data/J0613-0200_20cm.txt'},
  'RM': {'10cm': '/Users/xiaoxue/Dropbox/23_PPA/3_PPA_new/ppa/ionFR_correction/J0613-0200_ionFR_10cm.txt',
   '20cm': 

In [17]:
#=====================================================#
#     M + EFAC + EQUAD                                #
#=====================================================#
k = 0
logw_all = []
for i,psrn in enumerate(PSR_NAME_LIST):  
    for subset in ["10cm","20cm"]:
        
        psr = ppa.Pulsar( PSR_DICT_LIST[ psrn ] , order=2,iono="noiono",subset=subset,nfreqs_dict={"10cm":0 , "20cm":0})
        array = ppa.Array([psr])
        if array.NPSR != 0:
            _lnlike = array.Generate_Lnlike_Function(adm_signal='none')

            def lnlike(x):
                l10_EFAC , l10_EQUAD  = np.array(x)
                return _lnlike( [l10_EFAC] , [l10_EQUAD] , [-3.] , [0.] ,1., -22. , -8. )

            l10_EFAC_lp , l10_EFAC_sp = ppa.gen_uniform_lnprior(-2,2)
            l10_EQUAD_lp , l10_EQUAD_sp = ppa.gen_uniform_lnprior(-8,2)

            def lnprior( x ):
                l10_EFAC , l10_EQUAD  =  x

                LP_1 = l10_EFAC_lp(l10_EFAC)
                LP_2 = l10_EQUAD_lp(l10_EQUAD)

                return  np.sum([LP_1,LP_2])


            init = np.array([l10_EFAC_sp() , l10_EQUAD_sp()  ])
            

            outdir = "SPA_Chain_noiono/white_"+psr.PSR_NAME+"_"+subset
            if os.path.isdir(outdir):
                pass
            else:
                print(psrn,subset)
                sampler = PTSampler(len(init),lnlike,lnprior,cov = np.diag(np.ones(len(init)))*0.01,\
                                    resume=False,outDir=outdir,verbose=True)
                sampler.sample(init,100000)

            chain  = np.loadtxt("SPA_Chain_noiono/white_"+psrn+"_"+subset+"/chain_1.txt",skiprows=5000)


            med1 = np.median( chain[:,0])
            min1 = med1 - np.quantile( chain[:,0],0.16 )
            max1 = np.quantile( chain[:,0],0.84 ) - med1 


            med2 = np.median( chain[:,1])
            min2 = med2 - np.quantile( chain[:,1],0.16 )
            max2 = np.quantile( chain[:,1],0.84 ) - med2



            logVol = np.log( 4 * 10   )
            logz0 = lnlike([0,-7.  ])
            logBF_white = logVol - np.log( np.mean(np.exp( -( chain[:,-4]-chain[:,-4].max() ) ) ) )  + chain[:,-4].max() - logz0
            logw_all.append(logBF_white)
            
            
            # with only psi12
            
            
            #print( k+1,'&',psrn,"& $%.2f$"%np.log10(np.median(psr.DPA_ERR[0])),"& $%.2f$"%np.log10(np.std(psr.DPA[0])) ,"& $%.2f^{+%.2f}_{-%.2f}$  & $% .2f^{+%.2f}_{-%.2f} $ &  & "%(med1,max1,min1,med2,max2,min2),"& %.1f"%(logBF_white)+"\\\\" )
            
            k += 1

In [18]:
print(logw_all)

[54106.81297912263, 1362933.0355105393, 14.96195422481722, 1695.2667320158862, 1.7045658866933024, 29.17681693431132, 1411.989358146677, 178.00642725945667, 6122.9842058472, 52.84412004297428, 497758.87534736056, -0.5370935824947196, 10602.295861542654, 1.174591377447939, 4985.247092260799, 20.667260476072784, 225.70776066604344, 23.479230162985402, 44.60523565142694, 1578.6029013935383, 37.605628816697276, 5239.181395205909, 39.06075442275056, 58599.109084544936, -0.8658041450989913, 42860.31618603739, 2857.5937220205337, 7854197.228067178, 340.91496589261635, 18374.85808868084, 73.37339929776945, 1951.1360013604217, 2533280.683446159, -0.14236097306411466, 27698.06670210164, 66.55203992103543, 9.70015840316887, 1380.304707471034, 4767.248574285501, 522521.01961607207, 18.308246365623916, 5606.105588877112, 19538.07898064516, 14694.041061511522, 20.64998727019156, 23463.02039688788, 3.131108290052282, 2662.011286949626]


In [19]:
with open("ppa/Parfile/spa_results.json",'r') as f:
    spa_results = json.load(f)

In [20]:
#=====================================================#
#     M + EFAC + EQUAD + Sred + Gamma                 #
#=====================================================#
k = 0
for i,psrn in enumerate(PSR_NAME_LIST):  
    for subset in ["10cm","20cm"]:
        psr = ppa.Pulsar( PSR_DICT_LIST[ psrn ],order=2,iono="noiono",subset=subset,nfreqs_dict={"10cm":30 , "20cm":30})
        array = ppa.Array([psr])
        if array.NPSR != 0:
            _lnlike = array.Generate_Lnlike_Function(adm_signal='none')

            def lnlike(x):
                l10_EFAC , l10_EQUAD , l10_S0red  ,Gamma = np.array(x)
                return _lnlike( [l10_EFAC] , [l10_EQUAD] , [l10_S0red] , [Gamma] ,1, -22 , -8 )

            l10_EFAC_lp , l10_EFAC_sp = ppa.gen_uniform_lnprior(-2,2)
            l10_EQUAD_lp , l10_EQUAD_sp = ppa.gen_uniform_lnprior(-8,2)
            l10_S0red_lp , l10_S0red_sp = ppa.gen_uniform_lnprior(-8,2)
            Gamma_lp    , Gamma_sp = ppa.gen_uniform_lnprior(-8,2)
            
            def lnprior( x ):
                l10_EFAC , l10_EQUAD , l10_S0red , Gamma =  x

                LP_1 = l10_EFAC_lp(l10_EFAC)
                LP_2 = l10_EQUAD_lp(l10_EQUAD)
                LP_3 = l10_S0red_lp(l10_S0red)
                LP_4 = Gamma_lp(Gamma)

                return  np.sum([LP_1,LP_2,LP_3,LP_4])

            

            init = np.array([l10_EFAC_sp() , l10_EQUAD_sp() , l10_S0red_sp() , Gamma_sp() ])

            outdir = "SPA_Chain_noiono/red30_"+psr.PSR_NAME+"_"+subset
            if os.path.isdir(outdir):
                pass
            else:
                print(psrn,subset)
                sampler = PTSampler(len(init),lnlike,lnprior,cov = np.diag(np.ones(len(init)))*0.01,\
                                    resume=False,outDir=outdir,verbose=True)
                sampler.sample(init,100000)

            chain  = np.loadtxt("SPA_Chain_noiono/red30_"+psrn+"_"+subset+"/chain_1.txt",skiprows=5000)


            med1 = np.median( chain[:,0])
            min1 = med1 - np.quantile( chain[:,0],0.16 )
            max1 = np.quantile( chain[:,0],0.84 ) - med1 


            med2 = np.median( chain[:,1])
            min2 = med2 - np.quantile( chain[:,1],0.16 )
            max2 = np.quantile( chain[:,1],0.84 ) - med2


            med3 = np.median( chain[:,2])
            min3 = med3 - np.quantile( chain[:,2],0.16 )
            max3 = np.quantile( chain[:,2],0.84 ) - med3


            med4 = np.median( chain[:,3])
            min4 = med4 - np.quantile( chain[:,3],0.16 )
            max4 = np.quantile( chain[:,3],0.84 ) - med4



            
            logVol = np.log( 4 * 10 * 10 * 10   )
            logz0 = lnlike([0,-7 , -7 , 0   ])
            logBF = logVol - np.log( np.mean(np.exp( -( chain[:,-4]-chain[:,-4].max() ) ) ) )  + chain[:,-4].max() - logz0
            
            corner.corner(chain[:,[0,1,2,3]],labels=[r"$\log_{10}$EF" , r"$\log_{10}$EQ" , r"$\log_{10}S_{\rm red}$",r"$\Gamma$"],show_titles=True);
            plt.suptitle(psrn+"  "+subset)
            plt.savefig("Figures/"+psrn+"_"+subset+"_noiono.jpg")
            plt.close()
            spa_results[psrn].update( { "noiono_"+subset:(med1,med2,med3,med4,(logBF-logw_all[k]))} )
            # with only psi12
            
            
            print( k+1,'&',psrn,"& %.2f"%np.log10(np.median(psr.DPA_ERR[0])),"& %.2f"%np.log10(np.std(psr.DPA[0])) ,\
                  "& $%.2f^{+%.2f}_{-%.2f}$  & $% .2f^{+%.2f}_{-%.2f} $ & $ % .2f^{+%.2f}_{-%.2f}$ & $% .2f^{+%.2f}_{-%.2f}$"%(med1,max1,min1,med2,max2,min2,med3,max3,min3,med4,max4,min4),\
                    "& %.1f"%(logBF-logw_all[k])+"\\\\" )
                    #"& %.1f"%(logBF)+"\\\\" )
            
            k += 1

1 & J0437-4715 & -2.72 & -1.61 & $0.72^{+0.03}_{-0.03}$  & $-1.80^{+0.02}_{-0.02} $ & $ -2.76^{+0.24}_{-0.54}$ & $-1.92^{+0.71}_{-1.38}$ & 26.4\\
2 & J0437-4715 & -3.03 & -1.41 & $1.39^{+0.01}_{-0.01}$  & $-5.22^{+1.87}_{-1.90} $ & $ -1.68^{+0.04}_{-0.04}$ & $-0.62^{+0.21}_{-0.26}$ & 705.2\\
3 & J0613-0200 & -0.84 & -0.83 & $-0.20^{+0.05}_{-0.06}$  & $-1.02^{+0.05}_{-0.06} $ & $ -5.18^{+2.43}_{-1.92}$ & $-1.89^{+2.71}_{-3.62}$ & 2.0\\
4 & J0613-0200 & -1.68 & -1.21 & $-0.02^{+0.04}_{-0.04}$  & $-1.47^{+0.02}_{-0.02} $ & $ -1.70^{+0.05}_{-0.05}$ & $-1.04^{+0.22}_{-0.25}$ & 179.3\\
5 & J0614-3329 & -1.12 & -1.21 & $-0.13^{+0.06}_{-0.09}$  & $-2.67^{+1.12}_{-3.49} $ & $ -5.13^{+1.94}_{-1.86}$ & $-2.75^{+3.12}_{-3.47}$ & -0.4\\
6 & J0711-6830 & -0.82 & -0.78 & $-0.14^{+0.02}_{-0.03}$  & $-1.23^{+0.05}_{-0.06} $ & $ -5.49^{+1.99}_{-1.66}$ & $-2.40^{+3.07}_{-3.49}$ & -0.7\\
7 & J0711-6830 & -1.34 & -1.17 & $-0.17^{+0.02}_{-0.02}$  & $-1.52^{+0.02}_{-0.02} $ & $ -1.90^{+0.06}_{-0.06}$ & $-1.1

In [21]:
with open("ppa/Parfile/spa_results.json",'w') as f:
    spa_results = json.dump(spa_results,f,indent=2)